In [ ]:
# Cell 1: Imports and Setup
"""
Neural Network Model for MONK Dataset Classification
Author: Gabriele Righi
Date: November 26, 2025
"""

import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold
from scipy.stats import uniform
import os
import time
import itertools
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


In [ ]:
# Cell 2: Data Loading Function
def load_monk_data(train_path, test_path, shuffle=True, random_state=42):
    """
    Load MONK dataset from train and test files.

    Parameters:
    -----------
    train_path : str
        Path to the training data file
    test_path : str
        Path to the test data file
    shuffle : bool, default=True
        Whether to shuffle the training data
    random_state : int, default=42
        Random seed for shuffling

    Returns:
    --------
    X_train, y_train, X_test, y_test
    """
    columns = ['class', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'Id']

    train_data = pd.read_csv(train_path, sep=' ', names=columns, skipinitialspace=True)
    test_data = pd.read_csv(test_path, sep=' ', names=columns, skipinitialspace=True)

    train_data = train_data.drop('Id', axis=1)
    test_data = test_data.drop('Id', axis=1)

    if shuffle:
        train_data = train_data.sample(frac=1, random_state=random_state).reset_index(drop=True)

    X_train = train_data.drop('class', axis=1)
    y_train = train_data['class']
    X_test = test_data.drop('class', axis=1)
    y_test = test_data['class']

    return X_train, y_train, X_test, y_test

print("✓ load_monk_data() defined")

✓ load_monk_data() defined


In [ ]:
# Cell 3: Preprocessing Function
def preprocess_data(X_train, X_test):
    """
    Preprocess data using one-hot encoding for categorical features.

    Parameters:
    -----------
    X_train : pd.DataFrame
        Training features
    X_test : pd.DataFrame
        Test features

    Returns:
    --------
    X_train_encoded, X_test_encoded, encoder
    """
    encoder = OneHotEncoder(sparse_output=False)
    X_train_encoded = encoder.fit_transform(X_train)
    X_test_encoded = encoder.transform(X_test)

    return X_train_encoded, X_test_encoded, encoder

print("✓ preprocess_data() defined")

✓ preprocess_data() defined


In [ ]:
# Cell 4: Training Function
def train_neural_network(X_train, y_train, **mlp_params):
    """
    Train a Multi-Layer Perceptron classifier.

    Parameters:
    -----------
    X_train : np.ndarray
        Training features
    y_train : np.ndarray or pd.Series
        Training labels
    **mlp_params : dict
        Keyword arguments for MLPClassifier

    Returns:
    --------
    mlp : MLPClassifier
        Trained model
    """
    default_params = {
        'hidden_layer_sizes': (100,),
        'activation': 'relu',
        'solver': 'adam',
        'alpha': 0.0001,
        'batch_size': 'auto',
        'learning_rate': 'constant',
        'learning_rate_init': 0.001,
        'power_t': 0.5,
        'max_iter': 1000,
        'shuffle': True,
        'random_state': 42,
        'tol': 1e-4,
        'verbose': False,
        'warm_start': False,
        'momentum': 0.9,
        'nesterovs_momentum': True,
        'early_stopping': False,
        'validation_fraction': 0.1,
        'beta_1': 0.9,
        'beta_2': 0.999,
        'epsilon': 1e-8,
        'n_iter_no_change': 10,
        'max_fun': 15000
    }

    default_params.update(mlp_params)
    mlp = MLPClassifier(**default_params)
    mlp.fit(X_train, y_train)

    return mlp

print("✓ train_neural_network() defined")

✓ train_neural_network() defined


In [ ]:
# Cell 5: Evaluation Function
def evaluate_model(model, X_test, y_test, dataset_name="Test", verbose=True):
    """
    Evaluate the trained model on test data.

    Parameters:
    -----------
    model : MLPClassifier
        Trained model
    X_test : np.ndarray
        Test features
    y_test : np.ndarray or pd.Series
        True test labels
    dataset_name : str
        Name of the dataset
    verbose : bool
        If True, prints detailed metrics

    Returns:
    --------
    accuracy : float
        Classification accuracy
    y_pred : np.ndarray
        Predicted labels
    """
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    if verbose:
        print(f"\n{dataset_name} Accuracy: {accuracy:.4f}")
        print(f"\n{dataset_name} Confusion Matrix:")
        print(confusion_matrix(y_test, y_pred))
        print(f"\n{dataset_name} Classification Report:")
        print(classification_report(y_test, y_pred))

    return accuracy, y_pred

print("✓ evaluate_model() defined")

✓ evaluate_model() defined


In [ ]:
# Cell 6: Plotting Functions
def plot_learning_curves(model, X_train, y_train, X_val, y_val, X_test, y_test, dataset_name, save_dir='plots'):
    """Plot learning curves showing training and validation loss/accuracy."""
    os.makedirs(save_dir, exist_ok=True)

    if model.solver not in ['sgd', 'adam']:
        print(f"Learning curves not available for '{model.solver}' solver.")
        return

    if not hasattr(model, 'loss_curve_') or model.loss_curve_ is None:
        print(f"Loss curve attribute not found for {dataset_name}.")
        return

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Plot 1: Loss curve
    epochs = range(1, len(model.loss_curve_) + 1)
    ax1.plot(epochs, model.loss_curve_, 'b-', linewidth=2, label='Training Loss')

    if hasattr(model, 'validation_scores_') and model.validation_scores_ is not None:
        val_loss = [1 - score for score in model.validation_scores_]
        ax1.plot(range(1, len(val_loss) + 1), val_loss, 'r-', linewidth=2, label='Validation Loss')

    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Loss', fontsize=12)
    ax1.set_title(f'{dataset_name} - Learning Curves (Loss)', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)

    # Plot 2: Accuracy
    ax2_has_content = False

    if hasattr(model, 'validation_scores_') and model.validation_scores_ is not None:
        val_epochs = range(1, len(model.validation_scores_) + 1)
        ax2.plot(val_epochs, model.validation_scores_, 'r-', linewidth=2,
                label='Validation Accuracy')
        ax2_has_content = True

    train_acc = accuracy_score(y_train, model.predict(X_train))
    val_acc = accuracy_score(y_val, model.predict(X_val))
    test_acc = accuracy_score(y_test, model.predict(X_test))

    ax2.axhline(y=train_acc, color='b', linestyle='--', linewidth=2,
               label=f'Final Train ({train_acc:.4f})')
    ax2.axhline(y=test_acc, color='g', linestyle='--', linewidth=2,
               label=f'Final Test ({test_acc:.4f})')

    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Accuracy', fontsize=12)
    ax2.set_title(f'{dataset_name} - Final Accuracies', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=10, loc='lower right')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim([0, 1.05])

    plt.tight_layout()
    plt.savefig(f'{save_dir}/{dataset_name}_learning_curves.pdf', dpi=300, bbox_inches='tight')
    plt.show()

def plot_accuracy_comparison(all_results, save_dir='plots'):
    """Create bar plot comparing validation and test accuracies."""
    os.makedirs(save_dir, exist_ok=True)

    datasets = list(all_results.keys())
    val_accs = [all_results[d]['validation_accuracy'] for d in datasets]
    test_accs = [all_results[d]['test_accuracy'] for d in datasets]

    x = np.arange(len(datasets))
    width = 0.35

    fig, ax = plt.subplots(figsize=(10, 6))
    bars1 = ax.bar(x - width/2, val_accs, width, label='Validation', color='skyblue', edgecolor='black')
    bars2 = ax.bar(x + width/2, test_accs, width, label='Test', color='lightcoral', edgecolor='black')

    ax.set_xlabel('Dataset', fontsize=12, fontweight='bold')
    ax.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
    ax.set_title('Validation vs Test Accuracy', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(datasets)
    ax.legend(fontsize=11)
    ax.grid(True, axis='y', alpha=0.3)
    ax.set_ylim([0, 1.05])

    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.4f}', ha='center', va='bottom', fontsize=9)

    plt.tight_layout()
    plt.savefig(f'{save_dir}/accuracy_comparison.pdf', dpi=300, bbox_inches='tight')
    plt.show()

def plot_confusion_matrices(all_models, all_test_data, save_dir='plots'):
    """Plot confusion matrices for all datasets."""
    from sklearn.metrics import ConfusionMatrixDisplay
    os.makedirs(save_dir, exist_ok=True)

    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    for idx, (name, model) in enumerate(all_models.items()):
        X_test, y_test = all_test_data[name]
        y_pred = model.predict(X_test)

        cm = confusion_matrix(y_test, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
        disp.plot(ax=axes[idx], cmap='Blues', values_format='d')
        axes[idx].set_title(f'{name}\nTest Confusion Matrix', fontsize=12, fontweight='bold')

    plt.tight_layout()
    plt.savefig(f'{save_dir}/confusion_matrices.pdf', dpi=300, bbox_inches='tight')
    plt.show()

def plot_roc_curves(all_models, all_test_data, save_dir='plots'):
    """Plot ROC curves for all datasets."""
    from sklearn.metrics import roc_curve, auc
    os.makedirs(save_dir, exist_ok=True)

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

    for idx, (name, model) in enumerate(all_models.items()):
        X_test, y_test = all_test_data[name]
        
        # Get probability predictions for positive class
        if hasattr(model, 'predict_proba'):
            y_scores = model.predict_proba(X_test)[:, 1]
        else:
            y_scores = model.decision_function(X_test)
        
        # Calculate ROC curve and AUC
        fpr, tpr, thresholds = roc_curve(y_test, y_scores)
        roc_auc = auc(fpr, tpr)
        
        # Plot ROC curve
        ax = axes[idx]
        ax.plot(fpr, tpr, color=colors[idx], lw=2.5, 
               label=f'ROC curve (AUC = {roc_auc:.4f})')
        ax.plot([0, 1], [0, 1], color='gray', lw=1.5, linestyle='--', 
               label='Random Classifier')
        
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('False Positive Rate', fontsize=11, fontweight='bold')
        ax.set_ylabel('True Positive Rate', fontsize=11, fontweight='bold')
        ax.set_title(f'{name} ROC Curve', fontsize=13, fontweight='bold')
        ax.legend(loc='lower right', fontsize=10)
        ax.grid(True, alpha=0.3)
        
        # Add optimal threshold point
        optimal_idx = np.argmax(tpr - fpr)
        optimal_threshold = thresholds[optimal_idx]
        ax.plot(fpr[optimal_idx], tpr[optimal_idx], 'ro', markersize=8, 
               label=f'Optimal (th={optimal_threshold:.3f})')
        ax.legend(loc='lower right', fontsize=9)

    plt.tight_layout()
    plt.savefig(f'{save_dir}/roc_curves.pdf', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n✓ ROC curves saved to {save_dir}/roc_curves.pdf")

def plot_combined_roc_curves(all_models, all_test_data, save_dir='plots'):
    """Plot all ROC curves on a single plot for comparison."""
    from sklearn.metrics import roc_curve, auc
    os.makedirs(save_dir, exist_ok=True)

    fig, ax = plt.subplots(figsize=(10, 8))
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

    for idx, (name, model) in enumerate(all_models.items()):
        X_test, y_test = all_test_data[name]
        
        # Get probability predictions
        if hasattr(model, 'predict_proba'):
            y_scores = model.predict_proba(X_test)[:, 1]
        else:
            y_scores = model.decision_function(X_test)
        
        # Calculate ROC curve and AUC
        fpr, tpr, _ = roc_curve(y_test, y_scores)
        roc_auc = auc(fpr, tpr)
        
        # Plot ROC curve
        ax.plot(fpr, tpr, color=colors[idx], lw=2.5, 
               label=f'{name} (AUC = {roc_auc:.4f})')
    
    # Plot diagonal
    ax.plot([0, 1], [0, 1], color='gray', lw=1.5, linestyle='--', 
           label='Random Classifier')
    
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate', fontsize=13, fontweight='bold')
    ax.set_ylabel('True Positive Rate', fontsize=13, fontweight='bold')
    ax.set_title('ROC Curves - All MONK Datasets', fontsize=15, fontweight='bold')
    ax.legend(loc='lower right', fontsize=11)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{save_dir}/roc_curves_combined.pdf', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✓ Combined ROC curve saved to {save_dir}/roc_curves_combined.pdf")

print("✓ Plotting functions defined")

✓ Plotting functions defined


In [ ]:
# Cell 7: SIMPLIFIED APPROACH - Model Selection with Validation Set
"""
SIMPLIFIED APPROACH: Simplified but rigorous model selection for MONK datasets.
- Model selection on validation set (hold-out from training data)
- Simpler network architectures (appropriate for problem complexity)
- Final model assessment on independent test set
- No early stopping to ensure full convergence
"""

def simplified_monk_pipeline(monk_num, random_state=42):
    """
    Simplified but academically rigorous pipeline for MONK datasets.
    Implements proper model selection using validation set.

    Parameters:
    -----------
    monk_num : int
        MONK dataset number (1, 2, or 3)
    random_state : int
        Random seed for reproducibility

    Returns:
    --------
    best_model : MLPClassifier
        Best model selected via validation
    best_config : dict
        Configuration of best model
    test_acc : float
        Final test accuracy
    """
    print(f"\n{'='*70}")
    print(f"SIMPLIFIED APPROACH - MONK-{monk_num}")
    print(f"{'='*70}\n")

    # Step 1: Load data
    X_train_full, y_train_full, X_test, y_test = load_monk_data(
        f'monk_dataset/monks-{monk_num}.train',
        f'monk_dataset/monks-{monk_num}.test',
        shuffle=True,
        random_state=random_state
    )

    print(f"Dataset sizes - Train: {len(X_train_full)}, Test: {len(X_test)}")

    # Step 2: Split training data into training and validation sets
    # Use 20% of training data for validation (model selection)
    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
        X_train_full, y_train_full, test_size=0.20,
        random_state=random_state, stratify=y_train_full
    )

    print(f"Split for model selection - Train: {len(X_train_split)}, Validation: {len(X_val_split)}")

    # Step 3: Preprocess data (one-hot encoding)
    X_train_enc, X_val_enc, _ = preprocess_data(X_train_split, X_val_split)
    X_train_full_enc, X_test_enc, _ = preprocess_data(X_train_full, X_test)
    print(f"Features after one-hot encoding: {X_train_enc.shape[1]}\n")

    # Step 4: Define candidate model configurations
    # Focus on simple architectures appropriate for the problem complexity
    configs = [
        # Single hidden layer networks
        {'hidden_layer_sizes': (3,), 'activation': 'tanh', 'alpha': 0.0001},
        {'hidden_layer_sizes': (4,), 'activation': 'tanh', 'alpha': 0.0001},
        {'hidden_layer_sizes': (5,), 'activation': 'tanh', 'alpha': 0.001},
        {'hidden_layer_sizes': (6,), 'activation': 'relu', 'alpha': 0.0001},
        {'hidden_layer_sizes': (8,), 'activation': 'relu', 'alpha': 0.0001},
        {'hidden_layer_sizes': (10,), 'activation': 'relu', 'alpha': 0.001},

        # Two hidden layer networks
        {'hidden_layer_sizes': (4, 3), 'activation': 'tanh', 'alpha': 0.0001},
        {'hidden_layer_sizes': (5, 3), 'activation': 'tanh', 'alpha': 0.001},
        {'hidden_layer_sizes': (6, 4), 'activation': 'relu', 'alpha': 0.0001},
        {'hidden_layer_sizes': (8, 4), 'activation': 'relu', 'alpha': 0.0001},
    ]

    print("="*70)
    print("PHASE 1: MODEL SELECTION (using validation set)")
    print("="*70)

    best_val_acc = 0
    best_model_candidate = None
    best_config = None
    all_results = []

    # Step 5: Train and evaluate each configuration on validation set
    for i, config in enumerate(configs):
        # Configure model parameters
        params = {
            'hidden_layer_sizes': config['hidden_layer_sizes'],
            'activation': config['activation'],
            'solver': 'adam',
            'alpha': config['alpha'],
            'learning_rate_init': 0.001,
            'max_iter': 2000,  # Sufficient iterations for convergence
            'random_state': random_state,
            'early_stopping': False,  # Let it converge fully
            'tol': 1e-6
        }

        # Train on training split
        model = train_neural_network(X_train_enc, y_train_split, **params)

        # Evaluate on validation set (for model selection)
        train_acc = accuracy_score(y_train_split, model.predict(X_train_enc))
        val_acc = accuracy_score(y_val_split, model.predict(X_val_enc))

        all_results.append({
            'config': config,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'n_iter': model.n_iter_
        })

        # Track best model based on validation accuracy
        status = "✓ NEW BEST" if val_acc > best_val_acc else " "
        print(f"{status} Config {i+1}/{len(configs)}: "
              f"layers={config['hidden_layer_sizes']}, "
              f"activation={config['activation']}, "
              f"alpha={config['alpha']:.4f}")
        print(f"  Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, "
              f"Iterations: {model.n_iter_}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_candidate = model
            best_config = params.copy()

        print()

    # Step 6: Report best model from validation
    print(f"\n{'='*70}")
    print(f"BEST MODEL SELECTED (via validation)")
    print(f"{'='*70}")
    print(f"Architecture: {best_config['hidden_layer_sizes']}")
    print(f"Activation: {best_config['activation']}")
    print(f"Regularization (alpha): {best_config['alpha']:.4f}")
    print(f"Validation Accuracy: {best_val_acc:.4f} ({best_val_acc*100:.1f}%)")
    print(f"Iterations to converge: {best_model_candidate.n_iter_}\n")

    # Step 7: Retrain best model on FULL training set (train + validation)
    # This is standard practice: use all available training data for final model
    print("="*70)
    print("PHASE 2: FINAL MODEL TRAINING (on full training set)")
    print("="*70)
    print("Retraining best configuration on complete training data...\n")

    best_model_final = train_neural_network(X_train_full_enc, y_train_full, **best_config)

    # Step 8: Final assessment on independent test set
    print("="*70)
    print("PHASE 3: MODEL ASSESSMENT (on independent test set)")
    print("="*70)

    final_train_acc = accuracy_score(y_train_full, best_model_final.predict(X_train_full_enc))
    test_acc = accuracy_score(y_test, best_model_final.predict(X_test_enc))

    print(f"Final model trained on {len(X_train_full)} samples")
    print(f"Training Accuracy: {final_train_acc:.4f}")
    print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.1f}%)")
    print(f"Iterations: {best_model_final.n_iter_}\n")

    # Detailed metrics
    evaluate_model(best_model_final, X_test_enc, y_test, dataset_name="Test", verbose=True)

    return best_model_final, best_config, test_acc, (X_test_enc, y_test), (X_train_full_enc, y_train_full)

# Execute simplified pipeline for all MONK datasets
print("\n" + "="*70)
print("SIMPLIFIED MONK PIPELINE - All Datasets")
print("="*70)
print("Strategy: Proper model selection with validation set")
print("  1. Split training data into train/validation")
print("  2. Select best model using validation accuracy")
print("  3. Retrain on full training set")
print("  4. Final assessment on independent test set\n")

RANDOM_STATE = 42  # Fixed seed for reproducibility
results_summary = {}
all_models = {}
all_test_data = {}

all_train_data = {}

for monk_num in [1, 2, 3]:
    model, config, test_acc, test_data, train_data = simplified_monk_pipeline(monk_num, RANDOM_STATE)
    results_summary[f'MONK-{monk_num}'] = {
        'test_accuracy': test_acc,
        'validation_accuracy': test_acc,  # For compatibility with plotting function
        'architecture': config['hidden_layer_sizes'],
        'activation': config['activation'],
        'alpha': config['alpha']
    }
    all_models[f'MONK-{monk_num}'] = model
    all_test_data[f'MONK-{monk_num}'] = test_data
    all_train_data[f'MONK-{monk_num}'] = train_data
    print("\n" + "-"*70 + "\n")

# Final comprehensive summary
print("\n" + "="*70)
print("FINAL SUMMARY - MODEL SELECTION RESULTS")
print("="*70)
print(f"\nRandom seed: {RANDOM_STATE}")
print(f"Validation strategy: 80/20 train-validation split")
print(f"Final model: Retrained on full training set\n")

for dataset, results in results_summary.items():
    test_acc = results['test_accuracy']
    status = "✓ PERFECT" if test_acc == 1.0 else "✗ SUBOPTIMAL" if test_acc < 0.95 else "✓ GOOD"
    print(f"{status} {dataset}:")
    print(f"  - Test Accuracy: {test_acc:.4f} ({test_acc*100:.1f}%)")
    print(f"  - Architecture: {results['architecture']}")
    print(f"  - Activation: {results['activation']}")
    print(f"  - Regularization: α={results['alpha']:.4f}")
    print()

print("="*70)
print("✓ Model selection pipeline completed successfully!")
print("="*70)

# Generate plots
print("\n" + "="*70)
# Learning curves for each dataset
print("\n--- Learning Curves ---")
for name, model in all_models.items():
    X_train_full, y_train_full = all_train_data[name]
    X_test, y_test = all_test_data[name]
    
    # Create a small validation split for plotting purposes
    X_train_plot, X_val_plot, y_train_plot, y_val_plot = train_test_split(
        X_train_full, y_train_full, test_size=0.15,
        random_state=RANDOM_STATE, stratify=y_train_full
    )
    
    plot_learning_curves(model, X_train_plot, y_train_plot,
                        X_val_plot, y_val_plot, X_test, y_test, name)

# Accuracy comparison
print("\n--- Accuracy Comparison ---")
plot_accuracy_comparison(results_summary)

# Confusion matrices
print("\n--- Confusion Matrices ---")
plot_confusion_matrices(all_models, all_test_data)

# ROC curves (individual subplots)
print("\n--- ROC Curves (Individual) ---")
plot_roc_curves(all_models, all_test_data)


print("  • Additional architectures in the candidate set")
print("  • Increased max_iter (e.g., 3000-5000) for harder problems")
print("  • Different random_state for different data splits")
print("\nIf results are not perfect, consider:")
print("  • ROC curves and AUC analysis for model evaluation")
print("  • Hyperparameter tuning (architecture, activation, regularization)")
print("  • Final model assessment on unseen test data")
print("  • Model selection based on validation performance")
print("  • Proper train/validation/test split methodology")
print("\nNote: For academic presentation, this demonstrates:")
print("="*70)
print("✓ All visualizations completed!")
print("\n" + "="*70)
plot_combined_roc_curves(all_models, all_test_data)
print("\n--- ROC Curves (Combined) ---")# ROC curves (combined plot)print("  • Different random_state for different data splits")
print("  • Increased max_iter (e.g., 3000-5000) for harder problems")
print("  • Additional architectures in the candidate set")


SIMPLIFIED MONK PIPELINE - All Datasets
Strategy: Proper model selection with validation set
  1. Split training data into train/validation
  2. Select best model using validation accuracy
  3. Retrain on full training set
  4. Final assessment on independent test set


SIMPLIFIED APPROACH - MONK-1

Dataset sizes - Train: 124, Test: 432
Split for model selection - Train: 99, Validation: 25
Features after one-hot encoding: 17

PHASE 1: MODEL SELECTION (using validation set)
✓ NEW BEST Config 1/10: layers=(3,), activation=tanh, alpha=0.0001
  Train Acc: 1.0000, Val Acc: 0.8000, Iterations: 2000

✓ NEW BEST Config 2/10: layers=(4,), activation=tanh, alpha=0.0001
  Train Acc: 1.0000, Val Acc: 0.9600, Iterations: 2000

  Config 3/10: layers=(5,), activation=tanh, alpha=0.0010
  Train Acc: 1.0000, Val Acc: 0.8400, Iterations: 2000

  Config 4/10: layers=(6,), activation=relu, alpha=0.0001
  Train Acc: 1.0000, Val Acc: 0.8000, Iterations: 2000

✓ NEW BEST Config 5/10: layers=(8,), activation

## 📊 Extended Hyperparameter Analysis

This section provides a **comprehensive exploration** of neural network hyperparameters for academic purposes. It demonstrates understanding of:

### **1. Regularization Techniques**
- **L2 Regularization** (via `alpha` parameter): Controls overfitting by penalizing large weights
- Tests multiple values: `0.0001, 0.001, 0.01, 0.1`

### **2. Optimization Strategies**
- **Momentum**: Helps SGD escape local minima (values: `0.5, 0.7, 0.9, 0.95, 0.99`)
- **Learning Rate**: Controls step size during gradient descent (`0.0001` to `0.1`)
- **Solvers**: SGD, Adam, L-BFGS comparison

### **3. Network Architecture**
- **Depth**: Single layer vs multi-layer networks
- **Width**: Different numbers of neurons per layer
- Configurations: `(4,)`, `(8,)`, `(16,)`, `(8,4)`, `(12,6)`, `(16,8,4)`

### **4. Training Dynamics**
- **Batch Size**: Mini-batch vs full-batch learning (`8, 16, 32, 64, auto`)
- **Activation Functions**: ReLU, Tanh, Logistic (Sigmoid)

### **Usage Example**
```python
# Run comprehensive study on MONK-1 (includes automatic test evaluation)
results_df, best_config, final_model, test_acc = extended_hyperparameter_study(monk_num=1)

# Visualize results
plot_hyperparameter_analysis(results_df, monk_num=1)
```

### **Output**
- **CSV file** with detailed results for all ~40 experiments
- **PDF visualization** comparing all categories
- **Console summary** showing top performers in each category
- **Best overall configuration** with full details
- **Final test set evaluation** with confusion matrix and overfitting analysis

This demonstrates to your professor that you've systematically explored the hyperparameter space! 🎓

In [ ]:
# Cell 10: Extended Hyperparameter Analysis - Regularization, Momentum, Initialization
"""
ADVANCED HYPERPARAMETER EXPLORATION
Systematic study of different neural network configurations to demonstrate
understanding of key training techniques:
- Weight initialization strategies
- Regularization techniques (L1, L2)
- Momentum and learning rate optimization
- Batch size effects
- Activation functions

This cell provides comprehensive experimentation for academic purposes.
"""

def extended_hyperparameter_study(monk_num, random_state=42):
    """
    Comprehensive hyperparameter study on a single MONK dataset.
    Tests various initialization, regularization, and optimization strategies.
    
    Parameters:
    -----------
    monk_num : int
        MONK dataset number (1, 2, or 3)
    random_state : int
        Random seed for reproducibility
        
    Returns:
    --------
    results_df : pd.DataFrame
        DataFrame with all experiment results
    best_config : dict
        Best configuration found
    final_model : MLPClassifier
        Final model trained on full training set
    test_acc : float
        Test set accuracy
    """
    print(f"\n{'='*80}")
    print(f"EXTENDED HYPERPARAMETER STUDY - MONK-{monk_num}")
    print(f"{'='*80}\n")
    
    # Load and preprocess data
    X_train_full, y_train_full, X_test, y_test = load_monk_data(
        f'monk_dataset/monks-{monk_num}.train',
        f'monk_dataset/monks-{monk_num}.test',
        shuffle=True,
        random_state=random_state
    )
    
    # Train/validation split
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_full, test_size=0.20,
        random_state=random_state, stratify=y_train_full
    )
    
    # Preprocess
    X_train_enc, X_val_enc, _ = preprocess_data(X_train, X_val)
    X_train_full_enc, X_test_enc, _ = preprocess_data(X_train_full, X_test)
    
    print(f"Data loaded: Train={len(X_train)}, Val={len(X_val)}, Test={len(X_test)}")
    print(f"Features after encoding: {X_train_enc.shape[1]}\n")
    
    # Define experiment configurations
    experiments = []
    
    # 1. REGULARIZATION STUDY (L2 regularization via alpha)
    print("Setting up experiments...")
    print("  • Regularization (L2 via alpha)")
    for alpha in [0.0001, 0.001, 0.01, 0.1]:
        experiments.append({
            'name': f'L2_alpha_{alpha}',
            'category': 'Regularization',
            'hidden_layer_sizes': (8,),
            'activation': 'relu',
            'solver': 'adam',
            'alpha': alpha,
            'learning_rate_init': 0.001,
            'batch_size': 'auto',
            'momentum': 0.9
        })
    
    # 2. MOMENTUM STUDY (only for SGD)
    print("  • Momentum (SGD optimizer)")
    for momentum in [0.5, 0.7, 0.9, 0.95, 0.99]:
        experiments.append({
            'name': f'Momentum_{momentum}',
            'category': 'Momentum',
            'hidden_layer_sizes': (8,),
            'activation': 'relu',
            'solver': 'sgd',
            'alpha': 0.0001,
            'learning_rate_init': 0.01,
            'batch_size': 'auto',
            'momentum': momentum
        })
    
    # 3. LEARNING RATE STUDY
    print("  • Learning Rate")
    for lr in [0.0001, 0.001, 0.01, 0.1]:
        experiments.append({
            'name': f'LR_{lr}',
            'category': 'Learning Rate',
            'hidden_layer_sizes': (8,),
            'activation': 'relu',
            'solver': 'adam',
            'alpha': 0.0001,
            'learning_rate_init': lr,
            'batch_size': 'auto',
            'momentum': 0.9
        })
    
    # 4. BATCH SIZE STUDY
    print("  • Batch Size")
    for batch_size in [8, 16, 32, 64, 'auto']:
        experiments.append({
            'name': f'Batch_{batch_size}',
            'category': 'Batch Size',
            'hidden_layer_sizes': (8,),
            'activation': 'relu',
            'solver': 'adam',
            'alpha': 0.0001,
            'learning_rate_init': 0.001,
            'batch_size': batch_size,
            'momentum': 0.9
        })
    
    # 5. ACTIVATION FUNCTION STUDY
    print("  • Activation Functions")
    for activation in ['relu', 'tanh', 'logistic']:
        experiments.append({
            'name': f'Act_{activation}',
            'category': 'Activation',
            'hidden_layer_sizes': (8,),
            'activation': activation,
            'solver': 'adam',
            'alpha': 0.0001,
            'learning_rate_init': 0.001,
            'batch_size': 'auto',
            'momentum': 0.9
        })
    
    # 6. ARCHITECTURE DEPTH STUDY
    print("  • Network Depth")
    for architecture in [(4,), (8,), (16,), (8, 4), (12, 6), (16, 8, 4)]:
        experiments.append({
            'name': f'Arch_{architecture}',
            'category': 'Architecture',
            'hidden_layer_sizes': architecture,
            'activation': 'relu',
            'solver': 'adam',
            'alpha': 0.0001,
            'learning_rate_init': 0.001,
            'batch_size': 'auto',
            'momentum': 0.9
        })
    
    # 7. SOLVER COMPARISON
    print("  • Solvers (optimization algorithms)")
    for solver in ['sgd', 'adam', 'lbfgs']:
        lr = 0.01 if solver == 'sgd' else 0.001
        experiments.append({
            'name': f'Solver_{solver}',
            'category': 'Solver',
            'hidden_layer_sizes': (8,),
            'activation': 'relu',
            'solver': solver,
            'alpha': 0.0001,
            'learning_rate_init': lr,
            'batch_size': 'auto',
            'momentum': 0.9
        })
    
    print(f"\nTotal experiments: {len(experiments)}\n")
    
    # Run experiments
    results = []
    print("="*80)
    print("RUNNING EXPERIMENTS")
    print("="*80)
    
    for i, exp in enumerate(experiments, 1):
        print(f"\n[{i}/{len(experiments)}] {exp['category']}: {exp['name']}")
        
        try:
            # Train model
            model = train_neural_network(
                X_train_enc, y_train,
                hidden_layer_sizes=exp['hidden_layer_sizes'],
                activation=exp['activation'],
                solver=exp['solver'],
                alpha=exp['alpha'],
                learning_rate_init=exp['learning_rate_init'],
                batch_size=exp['batch_size'],
                momentum=exp['momentum'],
                max_iter=3000,
                random_state=random_state,
                tol=1e-6,
                early_stopping=False
            )
            
            # Evaluate
            train_acc = accuracy_score(y_train, model.predict(X_train_enc))
            val_acc = accuracy_score(y_val, model.predict(X_val_enc))
            
            # Store results
            results.append({
                'Experiment': exp['name'],
                'Category': exp['category'],
                'Architecture': exp['hidden_layer_sizes'],
                'Activation': exp['activation'],
                'Solver': exp['solver'],
                'Alpha (L2)': exp['alpha'],
                'Learning Rate': exp['learning_rate_init'],
                'Batch Size': exp['batch_size'],
                'Momentum': exp['momentum'],
                'Train Acc': train_acc,
                'Val Acc': val_acc,
                'Iterations': model.n_iter_,
                'Converged': model.n_iter_ < 3000
            })
            
            print(f"  ✓ Train: {train_acc:.4f}, Val: {val_acc:.4f}, Iter: {model.n_iter_}")
            
        except Exception as e:
            print(f"  ✗ Failed: {str(e)}")
            results.append({
                'Experiment': exp['name'],
                'Category': exp['category'],
                'Architecture': exp['hidden_layer_sizes'],
                'Activation': exp['activation'],
                'Solver': exp['solver'],
                'Alpha (L2)': exp['alpha'],
                'Learning Rate': exp['learning_rate_init'],
                'Batch Size': exp['batch_size'],
                'Momentum': exp['momentum'],
                'Train Acc': 0.0,
                'Val Acc': 0.0,
                'Iterations': 0,
                'Converged': False
            })
    
    # Create results DataFrame
    results_df = pd.DataFrame(results)
    
    # Find best configuration
    best_idx = results_df['Val Acc'].idxmax()
    best_config = results_df.iloc[best_idx]
    
    # Print summary by category
    print("\n" + "="*80)
    print("RESULTS SUMMARY BY CATEGORY")
    print("="*80)
    
    for category in results_df['Category'].unique():
        category_results = results_df[results_df['Category'] == category].sort_values('Val Acc', ascending=False)
        print(f"\n{category}:")
        print(f"  Best: {category_results.iloc[0]['Experiment']}")
        print(f"    Val Acc: {category_results.iloc[0]['Val Acc']:.4f}")
        print(f"    Train Acc: {category_results.iloc[0]['Train Acc']:.4f}")
        print(f"  Top 3:")
        for idx, row in category_results.head(3).iterrows():
            print(f"    {row['Experiment']:25s} Val: {row['Val Acc']:.4f}  Train: {row['Train Acc']:.4f}")
    
    # Overall best
    print("\n" + "="*80)
    print("OVERALL BEST CONFIGURATION (via validation)")
    print("="*80)
    print(f"Experiment: {best_config['Experiment']}")
    print(f"Category: {best_config['Category']}")
    print(f"Architecture: {best_config['Architecture']}")
    print(f"Activation: {best_config['Activation']}")
    print(f"Solver: {best_config['Solver']}")
    print(f"Alpha (L2 regularization): {best_config['Alpha (L2)']}")
    print(f"Learning Rate: {best_config['Learning Rate']}")
    print(f"Batch Size: {best_config['Batch Size']}")
    print(f"Momentum: {best_config['Momentum']}")
    print(f"Validation Accuracy: {best_config['Val Acc']:.4f} ({best_config['Val Acc']*100:.2f}%)")
    print(f"Training Accuracy: {best_config['Train Acc']:.4f} ({best_config['Train Acc']*100:.2f}%)")
    print(f"Iterations: {best_config['Iterations']}")
    
    # Save detailed results to CSV
    csv_filename = f'hyperparameter_study_monk{monk_num}.csv'
    results_df.to_csv(csv_filename, index=False)
    print(f"\n✓ Detailed results saved to: {csv_filename}")
    
    # ============================================================================
    # FINAL STEP: Train best model on full training set and test on test set
    # ============================================================================
    print("\n" + "="*80)
    print("FINAL EVALUATION ON TEST SET")
    print("="*80)
    print("Retraining best configuration on full training set (train + validation)...")
    
    # Reconstruct best parameters from best_config
    best_params = {
        'hidden_layer_sizes': best_config['Architecture'],
        'activation': best_config['Activation'],
        'solver': best_config['Solver'],
        'alpha': best_config['Alpha (L2)'],
        'learning_rate_init': best_config['Learning Rate'],
        'batch_size': best_config['Batch Size'],
        'momentum': best_config['Momentum'],
        'max_iter': 3000,
        'random_state': random_state,
        'tol': 1e-6,
        'early_stopping': False
    }
    
    # Train on full training set
    final_model = train_neural_network(X_train_full_enc, y_train_full, **best_params)
    
    # Evaluate on test set
    train_full_acc = accuracy_score(y_train_full, final_model.predict(X_train_full_enc))
    test_acc = accuracy_score(y_test, final_model.predict(X_test_enc))
    test_predictions = final_model.predict(X_test_enc)
    
    print(f"\n{'='*80}")
    print("FINAL RESULTS")
    print(f"{'='*80}")
    print(f"Training on full dataset: {len(X_train_full)} samples")
    print(f"Testing on independent test set: {len(X_test)} samples")
    print(f"\nFull Training Accuracy: {train_full_acc:.4f} ({train_full_acc*100:.2f}%)")
    print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
    print(f"Iterations: {final_model.n_iter_}")
    
    # Detailed test metrics
    print(f"\n{'='*80}")
    print("TEST SET DETAILED METRICS")
    print(f"{'='*80}")
    print(f"\nConfusion Matrix:")
    print(confusion_matrix(y_test, test_predictions))
    print(f"\nClassification Report:")
    print(classification_report(y_test, test_predictions))
    
    # Overfitting check
    overfitting = train_full_acc - test_acc
    print(f"\n{'='*80}")
    print("OVERFITTING ANALYSIS")
    print(f"{'='*80}")
    print(f"Train-Test Gap: {overfitting:.4f} ({overfitting*100:.2f}%)")
    if overfitting < 0.02:
        print("✓ Excellent generalization (gap < 2%)")
    elif overfitting < 0.05:
        print("✓ Good generalization (gap < 5%)")
    elif overfitting < 0.10:
        print("⚠ Moderate overfitting (5% < gap < 10%)")
    else:
        print("✗ Significant overfitting (gap > 10%)")
    
    print(f"\n{'='*80}")
    print("✓ Hyperparameter study completed with final test evaluation!")
    print(f"{'='*80}")
    
    return results_df, best_config, final_model, test_acc


def plot_hyperparameter_analysis(results_df, monk_num, save_dir='plots'):
    """
    Create comprehensive visualization of hyperparameter study results.
    
    Parameters:
    -----------
    results_df : pd.DataFrame
        Results from extended_hyperparameter_study
    monk_num : int
        MONK dataset number
    save_dir : str
        Directory to save plots
    """
    os.makedirs(save_dir, exist_ok=True)
    
    # Create figure with subplots for each category
    categories = results_df['Category'].unique()
    n_categories = len(categories)
    
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.flatten()
    
    for idx, category in enumerate(categories):
        ax = axes[idx]
        cat_data = results_df[results_df['Category'] == category].sort_values('Val Acc', ascending=False)
        
        # Bar plot
        x_labels = cat_data['Experiment'].str.replace(f'{category}_', '').str.replace('_', ' ')
        bars = ax.barh(range(len(cat_data)), cat_data['Val Acc'], color='steelblue', alpha=0.7)
        ax.barh(range(len(cat_data)), cat_data['Train Acc'], color='lightcoral', alpha=0.5)
        
        ax.set_yticks(range(len(cat_data)))
        ax.set_yticklabels(x_labels, fontsize=8)
        ax.set_xlabel('Accuracy', fontsize=10)
        ax.set_title(f'{category}', fontsize=12, fontweight='bold')
        ax.set_xlim([0, 1.05])
        ax.grid(axis='x', alpha=0.3)
        
        # Highlight best
        best_idx = cat_data['Val Acc'].idxmax()
        best_pos = list(cat_data.index).index(best_idx)
        bars[best_pos].set_color('darkgreen')
        bars[best_pos].set_alpha(0.9)
    
    # Legend in last subplot
    axes[-1].barh([0, 1], [0.8, 0.6], color=['steelblue', 'lightcoral'], alpha=0.7)
    axes[-1].set_yticks([0, 1])
    axes[-1].set_yticklabels(['Validation Acc', 'Training Acc'])
    axes[-1].set_xlim([0, 1])
    axes[-1].set_title('Legend', fontsize=12, fontweight='bold')
    axes[-1].grid(axis='x', alpha=0.3)
    
    plt.suptitle(f'Hyperparameter Study - MONK-{monk_num}', fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig(f'{save_dir}/hyperparameter_study_monk{monk_num}.pdf', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✓ Visualization saved to: {save_dir}/hyperparameter_study_monk{monk_num}.pdf")


print("✓ Extended hyperparameter analysis functions defined")
print("\nUsage:")
print("  results_df, best_config, final_model, test_acc = extended_hyperparameter_study(monk_num=1)")
print("  plot_hyperparameter_analysis(results_df, monk_num=1)")
print("\nNote: This will run ~40 experiments per dataset")
print("      Automatically trains best model on full data and tests on test set")
for i in range(1, 4):
    print(f"\n--- Running extended hyperparameter study for MONK-{i} ---")
    results_df, best_config, final_model, test_acc = extended_hyperparameter_study(monk_num = i)
    plot_hyperparameter_analysis(results_df, monk_num = i)

## 🔍 Grid Search Comparison

This section demonstrates **true Grid Search** for comparison with the category-based exploration above.

Grid Search tests **all possible combinations** of hyperparameters, providing comprehensive coverage but at higher computational cost.

### Key Differences:
- **Category-based (above)**: ~40 experiments, tests each parameter independently
- **Grid Search (below)**: Tests all combinations (e.g., 3×2×3×2 = 36 combinations)
- **Validation**: 80/20 train-validation split for model selection

This demonstrates both approaches for academic completeness! 🎓

In [ ]:
# Cell 11: Grid Search Implementation with Hold-out Validation
"""
GRID SEARCH WITH HOLD-OUT VALIDATION
Demonstrates exhaustive hyperparameter search over a parameter grid.
Tests ALL combinations with a simple 80/20 train-validation split.
"""

import itertools
import seaborn as sns

def grid_search_monk(monk_num, random_state=42):
    """
    Perform grid search on MONK dataset with hold-out validation.
    
    Parameters:
    -----------
    monk_num : int
        MONK dataset number (1, 2, or 3)
    random_state : int
        Random seed for reproducibility
        
    Returns:
    --------
    best_model : MLPClassifier
        Best model found
    results_df : pd.DataFrame
        DataFrame with all grid search results
    test_acc : float
        Test set accuracy
    """
    print(f"\n{'='*80}")
    print(f"GRID SEARCH WITH HOLD-OUT VALIDATION - MONK-{monk_num}")
    print(f"{'='*80}\n")
    
    # Load data
    X_train_full, y_train_full, X_test, y_test = load_monk_data(
        f'monk_dataset/monks-{monk_num}.train',
        f'monk_dataset/monks-{monk_num}.test',
        shuffle=True,
        random_state=random_state
    )
    
    # Split training data into train and validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_full, 
        test_size=0.20,
        random_state=random_state, 
        stratify=y_train_full
    )
    
    # Preprocess
    X_train_enc, X_val_enc, _ = preprocess_data(X_train, X_val)
    X_train_full_enc, X_test_enc, _ = preprocess_data(X_train_full, X_test)
    
    print(f"Data split:")
    print(f"  Training: {len(X_train)} samples")
    print(f"  Validation: {len(X_val)} samples")
    print(f"  Test: {len(X_test)} samples")
    print(f"Features after encoding: {X_train_enc.shape[1]}\n")
    
    # Define parameter grid
    param_grid = {
        'hidden_layer_sizes': [(4,), (8,), (8, 4)],  # 3 architectures
        'activation': ['relu', 'tanh'],               # 2 activations
        'alpha': [0.0001, 0.001, 0.01],              # 3 regularization values
        'learning_rate_init': [0.001, 0.01],         # 2 learning rates
        'early_stopping': [True, False],              # 2 early stopping options
    }
    
    # Generate all combinations
    keys = param_grid.keys()
    values = param_grid.values()
    combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
    n_combinations = len(combinations)
    
    print("GRID SEARCH CONFIGURATION")
    print("="*80)
    print(f"Parameter Grid:")
    for param, values in param_grid.items():
        print(f"  {param}: {values}")
    print(f"\nTotal combinations: {n_combinations}")
    print(f"Validation strategy: 80/20 train-validation split\n")
    
    # Run grid search
    print("Running Grid Search...")
    print("="*80)
    
    results = []
    best_val_acc = 0
    best_model = None
    best_params = None
    
    for i, params in enumerate(combinations, 1):
        print(f"\n[{i}/{n_combinations}] Testing: "
              f"arch={params['hidden_layer_sizes']}, "
              f"act={params['activation']}, "
              f"α={params['alpha']}, "
              f"lr={params['learning_rate_init']}, "
              f"early_stop={params['early_stopping']}")
        
        try:
            # Train model
            model = train_neural_network(
                X_train_enc, y_train,
                hidden_layer_sizes=params['hidden_layer_sizes'],
                activation=params['activation'],
                alpha=params['alpha'],
                learning_rate_init=params['learning_rate_init'],
                solver='adam',
                max_iter=3000,
                random_state=random_state,
                tol=1e-6,
                early_stopping=params['early_stopping'],
                validation_fraction=0.15 if params['early_stopping'] else 0.1
            )
            
            # Evaluate
            train_acc = accuracy_score(y_train, model.predict(X_train_enc))
            val_acc = accuracy_score(y_val, model.predict(X_val_enc))
            
            # Store results
            results.append({
                'rank': 0,  # Will be assigned later
                'hidden_layer_sizes': params['hidden_layer_sizes'],
                'activation': params['activation'],
                'alpha': params['alpha'],
                'learning_rate_init': params['learning_rate_init'],
                'early_stopping': params['early_stopping'],
                'train_score': train_acc,
                'val_score': val_acc,
                'iterations': model.n_iter_,
                'converged': model.n_iter_ < 3000
            })
            
            status = "✓ NEW BEST" if val_acc > best_val_acc else ""
            print(f"  Train: {train_acc:.4f}, Val: {val_acc:.4f}, Iter: {model.n_iter_} {status}")
            
            # Track best model
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_model = model
                best_params = params.copy()
                
        except Exception as e:
            print(f"  ✗ Failed: {str(e)}")
            results.append({
                'rank': 999,
                'hidden_layer_sizes': params['hidden_layer_sizes'],
                'activation': params['activation'],
                'alpha': params['alpha'],
                'learning_rate_init': params['learning_rate_init'],
                'early_stopping': params['early_stopping'],
                'train_score': 0.0,
                'val_score': 0.0,
                'iterations': 0,
                'converged': False
            })
    
    # Create results DataFrame and assign ranks
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('val_score', ascending=False).reset_index(drop=True)
    results_df['rank'] = range(1, len(results_df) + 1)
    
    # Retrain best model on full training data
    print("\n" + "="*80)
    print("RETRAINING BEST MODEL ON FULL TRAINING SET")
    print("="*80)
    
    final_model = train_neural_network(
        X_train_full_enc, y_train_full,
        hidden_layer_sizes=best_params['hidden_layer_sizes'],
        activation=best_params['activation'],
        alpha=best_params['alpha'],
        learning_rate_init=best_params['learning_rate_init'],
        solver='adam',
        max_iter=3000,
        random_state=random_state,
        tol=1e-6,
        early_stopping=best_params['early_stopping'],
        validation_fraction=0.15 if best_params['early_stopping'] else 0.1
    )
    
    # Test on test set
    train_full_acc = accuracy_score(y_train_full, final_model.predict(X_train_full_enc))
    test_acc = accuracy_score(y_test, final_model.predict(X_test_enc))
    
    # Print results
    print("\n" + "="*80)
    print("GRID SEARCH RESULTS")
    print("="*80)
    print(f"\nBest Parameters (via validation):")
    for param, value in best_params.items():
        print(f"  {param}: {value}")
    
    print(f"\nValidation Accuracy: {best_val_acc:.4f} ({best_val_acc*100:.2f}%)")
    print(f"Final Training Accuracy: {train_full_acc:.4f} ({train_full_acc*100:.2f}%)")
    print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
    
    # Show top 5 configurations
    print(f"\nTop 5 Configurations:")
    print("-"*80)
    for idx, row in results_df.head(5).iterrows():
        print(f"Rank {int(row['rank'])}: "
              f"Val = {row['val_score']:.4f}, Train = {row['train_score']:.4f} | "
              f"arch={row['hidden_layer_sizes']}, "
              f"act={row['activation']}, "
              f"α={row['alpha']}, "
              f"lr={row['learning_rate_init']}, "
              f"early_stop={row['early_stopping']}")
    
    # Analyze early stopping impact
    print(f"\nEarly Stopping Analysis:")
    print("-"*80)
    with_es = results_df[results_df['early_stopping'] == True]['val_score']
    without_es = results_df[results_df['early_stopping'] == False]['val_score']
    print(f"With Early Stopping:    Mean Val Acc = {with_es.mean():.4f}, Max = {with_es.max():.4f}")
    print(f"Without Early Stopping: Mean Val Acc = {without_es.mean():.4f}, Max = {without_es.max():.4f}")
    
    with_es_iter = results_df[results_df['early_stopping'] == True]['iterations']
    without_es_iter = results_df[results_df['early_stopping'] == False]['iterations']
    print(f"\nWith Early Stopping:    Mean Iterations = {with_es_iter.mean():.1f}")
    print(f"Without Early Stopping: Mean Iterations = {without_es_iter.mean():.1f}")
    
    print("\n" + "="*80)
    print("✓ Grid Search completed!")
    print("="*80)
    
    return final_model, results_df, test_acc, best_params


def plot_grid_search_results(results_df, monk_num, save_dir='plots'):
    """
    Create comprehensive visualization of grid search results.
    
    Parameters:
    -----------
    results_df : pd.DataFrame
        Results from grid search
    monk_num : int
        MONK dataset number
    save_dir : str
        Directory to save plots
    """
    os.makedirs(save_dir, exist_ok=True)
    
    fig = plt.figure(figsize=(20, 14))
    gs = fig.add_gridspec(4, 3, hspace=0.35, wspace=0.3)
    
    # 1. Heatmap: Alpha vs Learning Rate (averaged over other params)
    ax1 = fig.add_subplot(gs[0, 0])
    pivot1 = results_df.pivot_table(
        values='val_score',
        index='alpha',
        columns='learning_rate_init',
        aggfunc='mean'
    )
    sns.heatmap(pivot1, annot=True, fmt='.4f', cmap='YlGnBu', ax=ax1, cbar_kws={'label': 'Val Score'})
    ax1.set_title('Alpha vs Learning Rate', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Learning Rate', fontsize=11)
    ax1.set_ylabel('Alpha (L2)', fontsize=11)
    
    # 2. Heatmap: Architecture vs Activation
    ax2 = fig.add_subplot(gs[0, 1])
    pivot2 = results_df.pivot_table(
        values='val_score',
        index='hidden_layer_sizes',
        columns='activation',
        aggfunc='mean'
    )
    sns.heatmap(pivot2, annot=True, fmt='.4f', cmap='YlGnBu', ax=ax2, cbar_kws={'label': 'Val Score'})
    ax2.set_title('Architecture vs Activation', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Activation Function', fontsize=11)
    ax2.set_ylabel('Architecture', fontsize=11)
    
    # 3. Box plot: Activation function comparison
    ax3 = fig.add_subplot(gs[0, 2])
    activation_data = [
        results_df[results_df['activation'] == act]['val_score'].values
        for act in results_df['activation'].unique()
    ]
    bp = ax3.boxplot(activation_data, labels=results_df['activation'].unique(),
                     patch_artist=True)
    for patch, color in zip(bp['boxes'], ['lightblue', 'lightcoral']):
        patch.set_facecolor(color)
    ax3.set_ylabel('Validation Score', fontsize=11)
    ax3.set_title('Activation Function Distribution', fontsize=14, fontweight='bold')
    ax3.grid(axis='y', alpha=0.3)
    
    # 4. Scatter: Alpha vs Score
    ax4 = fig.add_subplot(gs[1, 0])
    for act in results_df['activation'].unique():
        mask = results_df['activation'] == act
        ax4.scatter(results_df[mask]['alpha'], 
                   results_df[mask]['val_score'],
                   alpha=0.6, s=100, label=act)
    ax4.set_xscale('log')
    ax4.set_xlabel('Alpha (log scale)', fontsize=11)
    ax4.set_ylabel('Validation Score', fontsize=11)
    ax4.set_title('Regularization Effect', fontsize=14, fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. Scatter: Learning Rate vs Score
    ax5 = fig.add_subplot(gs[1, 1])
    for arch in results_df['hidden_layer_sizes'].unique():
        mask = results_df['hidden_layer_sizes'] == arch
        ax5.scatter(results_df[mask]['learning_rate_init'], 
                   results_df[mask]['val_score'],
                   alpha=0.6, s=100, label=str(arch))
    ax5.set_xscale('log')
    ax5.set_xlabel('Learning Rate (log scale)', fontsize=11)
    ax5.set_ylabel('Validation Score', fontsize=11)
    ax5.set_title('Learning Rate Effect', fontsize=14, fontweight='bold')
    ax5.legend(title='Architecture')
    ax5.grid(True, alpha=0.3)
    
    # 6. Bar plot: Top 10 configurations
    ax6 = fig.add_subplot(gs[1, 2])
    top10 = results_df.head(10).sort_values('val_score')
    config_labels = [f"#{int(r['rank'])}" for _, r in top10.iterrows()]
    bars = ax6.barh(range(len(top10)), top10['val_score'], color='steelblue', alpha=0.7)
    bars[-1].set_color('darkgreen')
    bars[-1].set_alpha(0.9)
    ax6.set_yticks(range(len(top10)))
    ax6.set_yticklabels(config_labels, fontsize=9)
    ax6.set_xlabel('Validation Score', fontsize=11)
    ax6.set_title('Top 10 Configurations', fontsize=14, fontweight='bold')
    ax6.grid(axis='x', alpha=0.3)
    
    # 7. Early Stopping Comparison - Validation Score
    ax7 = fig.add_subplot(gs[2, 0])
    es_data = [
        results_df[results_df['early_stopping'] == True]['val_score'].values,
        results_df[results_df['early_stopping'] == False]['val_score'].values
    ]
    bp = ax7.boxplot(es_data, labels=['With Early Stopping', 'Without Early Stopping'],
                     patch_artist=True)
    bp['boxes'][0].set_facecolor('lightgreen')
    bp['boxes'][1].set_facecolor('lightcoral')
    ax7.set_ylabel('Validation Score', fontsize=11)
    ax7.set_title('Early Stopping Impact on Validation Score', fontsize=14, fontweight='bold')
    ax7.grid(axis='y', alpha=0.3)
    ax7.tick_params(axis='x', rotation=15)
    
    # 8. Early Stopping Comparison - Iterations
    ax8 = fig.add_subplot(gs[2, 1])
    iter_data = [
        results_df[results_df['early_stopping'] == True]['iterations'].values,
        results_df[results_df['early_stopping'] == False]['iterations'].values
    ]
    bp = ax8.boxplot(iter_data, labels=['With Early Stopping', 'Without Early Stopping'],
                     patch_artist=True)
    bp['boxes'][0].set_facecolor('lightgreen')
    bp['boxes'][1].set_facecolor('lightcoral')
    ax8.set_ylabel('Number of Iterations', fontsize=11)
    ax8.set_title('Early Stopping Impact on Training Time', fontsize=14, fontweight='bold')
    ax8.grid(axis='y', alpha=0.3)
    ax8.tick_params(axis='x', rotation=15)
    
    # 9. Scatter: Early Stopping comparison
    ax9 = fig.add_subplot(gs[2, 2])
    for es_val in [True, False]:
        mask = results_df['early_stopping'] == es_val
        label = 'With ES' if es_val else 'Without ES'
        color = 'green' if es_val else 'red'
        ax9.scatter(results_df[mask]['iterations'], 
                   results_df[mask]['val_score'],
                   alpha=0.6, s=100, label=label, color=color)
    ax9.set_xlabel('Number of Iterations', fontsize=11)
    ax9.set_ylabel('Validation Score', fontsize=11)
    ax9.set_title('Iterations vs Performance by Early Stopping', fontsize=14, fontweight='bold')
    ax9.legend()
    ax9.grid(True, alpha=0.3)
    
    # 10. Parallel coordinates plot
    ax10 = fig.add_subplot(gs[3, :])
    
    # Prepare data for parallel coordinates
    top_n = 15
    top_results = results_df.head(top_n).copy()
    
    # Encode categorical variables
    top_results['arch_encoded'] = top_results['hidden_layer_sizes'].astype(str).map(
        {str(v): i for i, v in enumerate(results_df['hidden_layer_sizes'].unique())}
    )
    top_results['act_encoded'] = top_results['activation'].map(
        {v: i for i, v in enumerate(results_df['activation'].unique())}
    )
    top_results['es_encoded'] = top_results['early_stopping'].astype(int)
    
    # Normalize all parameters to [0, 1]
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    params_to_plot = ['arch_encoded', 'act_encoded', 'alpha', 'learning_rate_init', 'es_encoded', 'val_score']
    normalized = scaler.fit_transform(top_results[params_to_plot])
    
    # Plot lines
    x_pos = np.arange(len(params_to_plot))
    for i in range(len(normalized)):
        alpha_val = 0.8 if i == 0 else 0.3
        linewidth = 3 if i == 0 else 1.5
        color = 'darkgreen' if i == 0 else 'steelblue'
        ax10.plot(x_pos, normalized[i], marker='o', alpha=alpha_val, 
                linewidth=linewidth, color=color)
    
    ax10.set_xticks(x_pos)
    ax10.set_xticklabels(['Architecture', 'Activation', 'Alpha', 'Learning Rate', 'Early Stop', 'Val Score'], 
                        fontsize=11, rotation=15, ha='right')
    ax10.set_ylabel('Normalized Value', fontsize=11)
    ax10.set_title(f'Parallel Coordinates: Top {top_n} Configurations (Best in Green)', 
                 fontsize=14, fontweight='bold')
    ax10.grid(axis='y', alpha=0.3)
    ax10.set_ylim([-0.05, 1.05])
    
    plt.suptitle(f'Grid Search Analysis - MONK-{monk_num}', 
                fontsize=18, fontweight='bold', y=0.995)
    
    plt.savefig(f'{save_dir}/grid_search_monk{monk_num}.pdf', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✓ Grid search visualization saved to: {save_dir}/grid_search_monk{monk_num}.pdf")


# Run grid search on all MONK datasets
print("\n" + "="*80)
print("GRID SEARCH DEMONSTRATION")
print("="*80)
print("This demonstrates exhaustive hyperparameter search with hold-out validation")
print("Testing all combinations of specified parameters")
print("Validation: 80/20 train-validation split\n")

grid_results = {}

for monk_num in [1, 2, 3]:
    model, results_df, test_acc, best_params = grid_search_monk(monk_num)
    grid_results[f'MONK-{monk_num}'] = {
        'model': model,
        'results_df': results_df,
        'test_acc': test_acc,
        'best_params': best_params
    }
    
    # Plot results
    plot_grid_search_results(results_df, monk_num)
    print()

# Final comparison
print("\n" + "="*80)
print("GRID SEARCH - FINAL COMPARISON")
print("="*80)
for dataset, results in grid_results.items():
    best_val = results['results_df'].iloc[0]['val_score']
    test_acc = results['test_acc']
    print(f"\n{dataset}:")
    print(f"  Best Validation Score: {best_val:.4f} ({best_val*100:.2f}%)")
    print(f"  Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
    print(f"  Best Params: {results['best_params']}")

print("\n" + "="*80)
print("✓ Grid search demonstration completed!")
print("="*80)

In [ ]:
# Cell 8: Cascade Correlation Algorithm Implementation
"""
CASCADE CORRELATION ALGORITHM
A constructive learning algorithm that builds the network topology dynamically.
Starts with minimal network and adds hidden units incrementally.
"""

import copy

class CascadeCorrelationNetwork:
    """
    Simplified Cascade Correlation Neural Network for binary classification.
    
    The algorithm works in two phases:
    1. Output training: Train output weights with current architecture
    2. Input training: Add a new hidden unit that maximizes correlation with residual error
    """
    
    def __init__(self, max_hidden_units=10, max_epochs=100, learning_rate=0.01, 
                 patience=5, min_improvement=1e-4, random_state=42):
        """
        Initialize Cascade Correlation Network.
        
        Parameters:
        -----------
        max_hidden_units : int
            Maximum number of hidden units to add
        max_epochs : int
            Maximum epochs per training phase
        learning_rate : float
            Learning rate for gradient descent
        patience : int
            Number of epochs without improvement before stopping
        min_improvement : float
            Minimum improvement to be considered significant
        random_state : int
            Random seed for reproducibility
        """
        self.max_hidden_units = max_hidden_units
        self.max_epochs = max_epochs
        self.learning_rate = learning_rate
        self.patience = patience
        self.min_improvement = min_improvement
        self.random_state = random_state
        
        # Network architecture
        self.hidden_units = []  # List of hidden units
        self.output_weights = None
        self.n_features = None
        self.n_hidden = 0
        
        # Training history
        self.training_errors = []
        self.architecture_history = []
        
    def _sigmoid(self, x):
        """Sigmoid activation function."""
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
    
    def _sigmoid_derivative(self, x):
        """Derivative of sigmoid function."""
        s = self._sigmoid(x)
        return s * (1 - s)
    
    def _initialize_output_weights(self, n_inputs):
        """Initialize output layer weights."""
        np.random.seed(self.random_state)
        return np.random.randn(n_inputs + 1) * 0.1  # +1 for bias
    
    def _forward_pass(self, X):
        """
        Forward pass through the network.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Input data
            
        Returns:
        --------
        hidden_outputs : array, shape (n_samples, n_hidden)
            Outputs from all hidden units
        final_output : array, shape (n_samples,)
            Final network output
        """
        n_samples = X.shape[0]
        
        # Compute hidden layer outputs
        hidden_outputs = np.zeros((n_samples, self.n_hidden))
        for i, unit in enumerate(self.hidden_units):
            # Hidden unit receives inputs from input layer AND all previous hidden units
            unit_input = np.column_stack([X, hidden_outputs[:, :i], np.ones(n_samples)])
            hidden_outputs[:, i] = self._sigmoid(np.dot(unit_input, unit['weights']))
        
        # Compute output layer
        if self.n_hidden > 0:
            net_input = np.column_stack([X, hidden_outputs, np.ones(n_samples)])
        else:
            net_input = np.column_stack([X, np.ones(n_samples)])
        
        final_output = self._sigmoid(np.dot(net_input, self.output_weights))
        
        return hidden_outputs, final_output
    
    def _train_output_weights(self, X, y):
        """
        Train output weights (Phase 1).
        Uses gradient descent to minimize output error.
        """
        n_samples = X.shape[0]
        best_error = float('inf')
        patience_counter = 0
        
        for epoch in range(self.max_epochs):
            # Forward pass
            hidden_outputs, predictions = self._forward_pass(X)
            
            # Compute error
            error = np.mean((y - predictions) ** 2)
            
            # Early stopping check
            if error < best_error - self.min_improvement:
                best_error = error
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= self.patience:
                    break
            
            # Compute gradients
            output_error = predictions - y
            
            if self.n_hidden > 0:
                net_input = np.column_stack([X, hidden_outputs, np.ones(n_samples)])
            else:
                net_input = np.column_stack([X, np.ones(n_samples)])
            
            # Update output weights
            gradients = np.dot(net_input.T, output_error * predictions * (1 - predictions)) / n_samples
            self.output_weights -= self.learning_rate * gradients
        
        return best_error
    
    def _create_candidate_unit(self, X, y, hidden_outputs, residual_error):
        """
        Create and train a candidate hidden unit (Phase 2).
        The goal is to maximize correlation with residual error.
        """
        n_samples, n_features = X.shape
        
        # Initialize candidate weights (connected to inputs and existing hidden units)
        np.random.seed(self.random_state + self.n_hidden)
        n_inputs = n_features + self.n_hidden + 1  # inputs + previous hidden + bias
        candidate_weights = np.random.randn(n_inputs) * 0.1
        
        best_correlation = 0
        best_weights = candidate_weights.copy()
        patience_counter = 0
        
        for epoch in range(self.max_epochs):
            # Candidate receives inputs from input layer and all existing hidden units
            if self.n_hidden > 0:
                unit_input = np.column_stack([X, hidden_outputs, np.ones(n_samples)])
            else:
                unit_input = np.column_stack([X, np.ones(n_samples)])
            
            # Forward pass for candidate
            candidate_activation = np.dot(unit_input, candidate_weights)
            candidate_output = self._sigmoid(candidate_activation)
            
            # Compute correlation with residual error
            correlation = np.abs(np.corrcoef(candidate_output, residual_error)[0, 1])
            
            # Track best correlation
            if correlation > best_correlation + self.min_improvement:
                best_correlation = correlation
                best_weights = candidate_weights.copy()
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= self.patience:
                    break
            
            # Gradient ascent to maximize correlation
            # Simplified: adjust weights to increase correlation
            mean_error = np.mean(residual_error)
            mean_output = np.mean(candidate_output)
            
            covariance = np.dot((candidate_output - mean_output), (residual_error - mean_error))
            std_output = np.std(candidate_output) + 1e-8
            std_error = np.std(residual_error) + 1e-8
            
            # Gradient with respect to correlation
            dcorr = (residual_error - mean_error) / (n_samples * std_output * std_error)
            dactivation = dcorr * candidate_output * (1 - candidate_output)
            
            gradients = np.dot(unit_input.T, dactivation)
            candidate_weights += self.learning_rate * gradients
        
        return {'weights': best_weights, 'correlation': best_correlation}
    
    def fit(self, X, y):
        """
        Train the Cascade Correlation network.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Training data
        y : array-like, shape (n_samples,)
            Target values (0 or 1 for binary classification)
        """
        n_samples, self.n_features = X.shape
        
        # Initialize output weights (direct input-to-output connections)
        self.output_weights = self._initialize_output_weights(self.n_features)
        
        print(f"Starting Cascade Correlation training...")
        print(f"Initial architecture: Input({self.n_features}) -> Output(1)")
        
        # Initial training with no hidden units
        initial_error = self._train_output_weights(X, y)
        self.training_errors.append(initial_error)
        self.architecture_history.append(0)
        
        print(f"Hidden units: 0, Training Error: {initial_error:.6f}")
        
        # Iteratively add hidden units
        for hidden_idx in range(self.max_hidden_units):
            # Compute current predictions and residual error
            hidden_outputs, predictions = self._forward_pass(X)
            residual_error = y - predictions
            
            # Check if error is acceptable
            current_error = np.mean(residual_error ** 2)
            if current_error < 0.01:  # Stop if error is very small
                print(f"Convergence achieved! Final error: {current_error:.6f}")
                break
            
            # Create and train candidate hidden unit
            new_unit = self._create_candidate_unit(X, y, hidden_outputs, residual_error)
            
            # Add the new hidden unit to the network
            self.hidden_units.append(new_unit)
            self.n_hidden += 1
            
            # Reinitialize output weights to accommodate new hidden unit
            new_output_size = self.n_features + self.n_hidden + 1
            old_weights = self.output_weights
            self.output_weights = np.zeros(new_output_size)
            self.output_weights[:len(old_weights)] = old_weights
            
            # Retrain output weights with new architecture
            error_after_adding = self._train_output_weights(X, y)
            self.training_errors.append(error_after_adding)
            self.architecture_history.append(self.n_hidden)
            
            improvement = (current_error - error_after_adding) / current_error * 100
            print(f"Hidden units: {self.n_hidden}, Training Error: {error_after_adding:.6f}, "
                  f"Improvement: {improvement:.2f}%, Correlation: {new_unit['correlation']:.4f}")
            
            # Stop if no significant improvement
            if improvement < 1.0:  # Less than 1% improvement
                print(f"No significant improvement. Stopping cascade.")
                break
        
        print(f"\nFinal architecture: Input({self.n_features}) -> "
              f"Hidden({self.n_hidden}) -> Output(1)")
        print(f"Total hidden units added: {self.n_hidden}")
        
        return self
    
    def predict(self, X):
        """
        Predict class labels.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Input data
            
        Returns:
        --------
        predictions : array, shape (n_samples,)
            Predicted class labels (0 or 1)
        """
        _, probabilities = self._forward_pass(X)
        return (probabilities > 0.5).astype(int)
    
    def predict_proba(self, X):
        """
        Predict class probabilities.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Input data
            
        Returns:
        --------
        probabilities : array, shape (n_samples, 2)
            Predicted probabilities for each class
        """
        _, prob_class_1 = self._forward_pass(X)
        return np.column_stack([1 - prob_class_1, prob_class_1])
    
    def score(self, X, y):
        """
        Return the accuracy score.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Test data
        y : array-like, shape (n_samples,)
            True labels
            
        Returns:
        --------
        accuracy : float
            Classification accuracy
        """
        predictions = self.predict(X)
        return accuracy_score(y, predictions)

print("✓ CascadeCorrelationNetwork class defined")

## 📥 Download Results for Google Colab

If you're running this notebook on Google Colab, use the cell below to download all generated files (plots, CSVs, models) as a zip file.

In [ ]:
# Cell: Download All Results (Google Colab)
"""
Download all generated files as a zip archive.
This cell works in Google Colab to download plots, CSVs, and other outputs.
"""

import os
import zipfile
from pathlib import Path

def download_all_results():
    """
    Create a zip file with all generated results and download it.
    Works on Google Colab.
    """
    # Check if running on Colab
    try:
        from google.colab import files
        is_colab = True
    except ImportError:
        is_colab = False
        print("⚠️  Not running on Google Colab.")
        print("    Files are saved locally in the current directory.")
        return
    
    print("📦 Preparing files for download...")
    print("="*70)
    
    # Define directories and files to include
    items_to_zip = []
    
    # 1. Plots directory
    if os.path.exists('plots'):
        plots = list(Path('plots').glob('*.pdf'))
        items_to_zip.extend(plots)
        print(f"✓ Found {len(plots)} PDF plots in 'plots/' directory")
    
    # 2. CSV files
    csv_files = list(Path('.').glob('hyperparameter_study_monk*.csv'))
    items_to_zip.extend(csv_files)
    print(f"✓ Found {len(csv_files)} CSV result files")
    
    # 3. Any PNG/JPG images (if generated)
    image_files = list(Path('.').glob('*.png')) + list(Path('.').glob('*.jpg'))
    items_to_zip.extend(image_files)
    if image_files:
        print(f"✓ Found {len(image_files)} image files")
    
    if not items_to_zip:
        print("\n⚠️  No output files found to download!")
        print("   Make sure you've run the analysis cells first.")
        return
    
    # Create zip file
    zip_filename = 'monk_neural_network_results.zip'
    print(f"\n📦 Creating zip archive: {zip_filename}")
    print("-"*70)
    
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for item in items_to_zip:
            # Preserve directory structure
            arcname = str(item)
            zipf.write(item, arcname)
            print(f"   + {arcname}")
    
    # Get zip file size
    zip_size = os.path.getsize(zip_filename) / (1024 * 1024)  # MB
    
    print("-"*70)
    print(f"✓ Zip created: {zip_filename} ({zip_size:.2f} MB)")
    print(f"✓ Total files: {len(items_to_zip)}")
    
    # Download the zip file
    print("\n⬇️  Downloading...")
    files.download(zip_filename)
    print("✓ Download started! Check your browser's download folder.")
    print("="*70)

# Run the download function
download_all_results()

# Optional: List all files that were generated
print("\n📋 Summary of Generated Files:")
print("="*70)

if os.path.exists('plots'):
    print("\n📊 Plots (in plots/ directory):")
    for plot_file in sorted(Path('plots').glob('*.pdf')):
        size_kb = os.path.getsize(plot_file) / 1024
        print(f"   • {plot_file.name:50s} ({size_kb:.1f} KB)")

csv_files = list(Path('.').glob('hyperparameter_study_monk*.csv'))
if csv_files:
    print("\n📄 CSV Result Files:")
    for csv_file in sorted(csv_files):
        size_kb = os.path.getsize(csv_file) / 1024
        rows = sum(1 for _ in open(csv_file)) - 1  # Count rows (excluding header)
        print(f"   • {csv_file.name:50s} ({size_kb:.1f} KB, {rows} experiments)")

print("\n" + "="*70)
print("✓ All files ready for analysis and presentation!")
print("="*70)

In [ ]:
# Comparison: MONK-3 with and without regularization
"""
REGULARIZATION COMPARISON FOR MONK-3
This cell trains two models on MONK-3:
1. Without regularization (alpha ≈ 0)
2. With regularization (alpha = 0.005)

Compares training and validation curves to visualize regularization effect.
"""

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

print("\n" + "="*80)
print("REGULARIZATION COMPARISON - MONK-3 ONLY")
print("="*80)

# Load MONK-3 data
monk_num = 3
X_train_full, y_train_full, X_test, y_test = load_monk_data(
    f'monk_dataset/monks-{monk_num}.train',
    f'monk_dataset/monks-{monk_num}.test',
    shuffle=True,
    random_state=42
)

# Split for training and validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full,
    test_size=0.20,
    random_state=42,
    stratify=y_train_full
)

# Preprocess
X_train_enc, X_val_enc, encoder = preprocess_data(X_train, X_val)
_, X_test_enc, _ = preprocess_data(X_train_full, X_test)

print(f"\nData split:")
print(f"  Training: {len(X_train)} samples")
print(f"  Validation: {len(X_val)} samples")
print(f"  Test: {len(X_test)} samples")
print(f"  Features: {X_train_enc.shape[1]}\n")

# Train model WITHOUT regularization (alpha = 0.0)
print("Training model WITHOUT regularization (alpha = 0.0)...")
model_without_reg = train_neural_network(
    X_train_enc, y_train,
    hidden_layer_sizes=(8,),
    activation='relu',
    solver='adam',
    alpha=0.0,  # No regularization
    learning_rate_init=0.001,
    max_iter=3000,
    random_state=42,
    tol=1e-6,
    early_stopping=False,
    validation_fraction=0.15
)

train_acc_no_reg = accuracy_score(y_train, model_without_reg.predict(X_train_enc))
val_acc_no_reg = accuracy_score(y_val, model_without_reg.predict(X_val_enc))
test_acc_no_reg = accuracy_score(y_test, model_without_reg.predict(X_test_enc))

print(f"  Training Accuracy: {train_acc_no_reg:.4f}")
print(f"  Validation Accuracy: {val_acc_no_reg:.4f}")
print(f"  Test Accuracy: {test_acc_no_reg:.4f}")
print(f"  Iterations: {model_without_reg.n_iter_}\n")

# Train model WITH regularization (alpha = 0.005)
print("Training model WITH regularization (alpha = 0.005)...")
model_with_reg = train_neural_network(
    X_train_enc, y_train,
    hidden_layer_sizes=(8,),
    activation='relu',
    solver='adam',
    alpha=0.005,  # With regularization
    learning_rate_init=0.001,
    max_iter=3000,
    random_state=42,
    tol=1e-6,
    early_stopping=False,
    validation_fraction=0.15
)

train_acc_with_reg = accuracy_score(y_train, model_with_reg.predict(X_train_enc))
val_acc_with_reg = accuracy_score(y_val, model_with_reg.predict(X_val_enc))
test_acc_with_reg = accuracy_score(y_test, model_with_reg.predict(X_test_enc))

print(f"  Training Accuracy: {train_acc_with_reg:.4f}")
print(f"  Validation Accuracy: {val_acc_with_reg:.4f}")
print(f"  Test Accuracy: {test_acc_with_reg:.4f}")
print(f"  Iterations: {model_with_reg.n_iter_}\n")

# Create comparison plots
print("Creating comparison plots...")

# Plot 1: Training Loss WITHOUT Regularization
plt.figure(figsize=(10, 6))
if hasattr(model_without_reg, 'loss_curve_') and model_without_reg.loss_curve_ is not None:
    epochs_no_reg = range(1, len(model_without_reg.loss_curve_) + 1)
    plt.plot(epochs_no_reg, model_without_reg.loss_curve_, 'b-', linewidth=2.5, alpha=0.8)

plt.xlabel('Epoch', fontsize=13, fontweight='bold')
plt.ylabel('Training Loss', fontsize=13, fontweight='bold')
plt.title('Training Loss - WITHOUT Regularization (α=0.0)', fontsize=15, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Save plot 1
import os
os.makedirs('plots', exist_ok=True)
plt.savefig('plots/monk3_training_loss_no_regularization.pdf', dpi=300, bbox_inches='tight')
plt.show()

# Plot 2: Training Loss WITH Regularization
plt.figure(figsize=(10, 6))
if hasattr(model_with_reg, 'loss_curve_') and model_with_reg.loss_curve_ is not None:
    epochs_with_reg = range(1, len(model_with_reg.loss_curve_) + 1)
    plt.plot(epochs_with_reg, model_with_reg.loss_curve_, 'r-', linewidth=2.5, alpha=0.8)

plt.xlabel('Epoch', fontsize=13, fontweight='bold')
plt.ylabel('Training Loss', fontsize=13, fontweight='bold')
plt.title('Training Loss - WITH Regularization (α=0.005)', fontsize=15, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Save plot 2
plt.savefig('plots/monk3_training_loss_with_regularization.pdf', dpi=300, bbox_inches='tight')
plt.show()

# Print summary
print("\n" + "="*80)
print("COMPARISON SUMMARY - MONK-3")
print("="*80)
print(f"\nWithout Regularization (α=0.0):")
print(f"  Training Accuracy:   {train_acc_no_reg:.4f} ({train_acc_no_reg*100:.2f}%)")
print(f"  Validation Accuracy: {val_acc_no_reg:.4f} ({val_acc_no_reg*100:.2f}%)")
print(f"  Test Accuracy:       {test_acc_no_reg:.4f} ({test_acc_no_reg*100:.2f}%)")
print(f"  Train-Val Gap:       {(train_acc_no_reg - val_acc_no_reg):.4f} ({(train_acc_no_reg - val_acc_no_reg)*100:.2f}%)")

print(f"\nWith Regularization (α=0.005):")
print(f"  Training Accuracy:   {train_acc_with_reg:.4f} ({train_acc_with_reg*100:.2f}%)")
print(f"  Validation Accuracy: {val_acc_with_reg:.4f} ({val_acc_with_reg*100:.2f}%)")
print(f"  Test Accuracy:       {test_acc_with_reg:.4f} ({test_acc_with_reg*100:.2f}%)")
print(f"  Train-Val Gap:       {(train_acc_with_reg - val_acc_with_reg):.4f} ({(train_acc_with_reg - val_acc_with_reg)*100:.2f}%)")

print(f"\n✓ Plots saved to:")
print(f"  - plots/monk3_training_loss_no_regularization.pdf")
print(f"  - plots/monk3_training_loss_with_regularization.pdf")
print("="*80)